# 🏍️ LossZero: Motorcycle Night Ride SegFormer-B2 Optimized

이 노트북은 **SegFormer-B2** 모델을 사용하여 야간 오토바이 주행 이미지의 시멘틱 세그멘테이션을 수행합니다.

### 🛠️ 주요 시나리오
- **모델**: SegFormer-B2 (Transformer 기반)
- **백본**: MiT-B2
- **사전 학습**: Cityscapes (도로 환경 특화)
- **최적화**: AdamW + FP16 Mixed Precision
- **손실 함수**: Weighted CrossEntropy (중요 객체 가중치 부여)

In [ ]:
import os
import cv2
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import albumentations as A
from albumentations.pytorch import ToTensorV2
from transformers import SegformerForSemanticSegmentation, SegformerConfig
from torch.amp import autocast, GradScaler
from tqdm.auto import tqdm

print(f"PyTorch version: {torch.__version__}")


## Colab 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [77]:
def get_device():
    if torch.cuda.is_available():
        return "cuda"

    return "cpu"

def num_worker():
    if torch.cuda.is_available():
        return os.cpu_count()

    return 0

# ⚙️ 설정 (Configuration)
#DATA_DIR = "/content/drive/MyDrive/motor_model"
DATA_DIR = os.path.expanduser("~/Projects/LossZero/data/Motorcycle Night Ride Dataset")
print("Detected Local Environment")

JSON_PATH = os.path.join(DATA_DIR, "COCO_motorcycle (pixel).json")
IMG_DIR = os.path.join(DATA_DIR, "images")

CFG = {
    "project": "LossZero",
    "model_name": "nvidia/segformer-b2-finetuned-cityscapes-1024-1024",
    "img_size": (352, 352),
    "batch_size": 4,
    "lr": 1e-4,
    "epochs": 25,
    "device": get_device(),
    "num_worker": num_worker()
}

print(f"Using device: {CFG['device']}")
print(f"Data directory: {DATA_DIR}")

Detected Local Environment
Using device: cpu
Data directory: /Users/jamesyang/Projects/LossZero/data/Motorcycle Night Ride Dataset


In [78]:
def create_mask_from_json(coco, img_id, img_info, id_to_idx):
    ann_ids = coco.getAnnIds(imgIds=img_id)
    anns = coco.loadAnns(ann_ids)
    mask = np.zeros((img_info['height'], img_info['width']), dtype=np.uint8)

    for ann in anns:
        cat_id = ann['category_id']
        if cat_id in id_to_idx:
            cls_idx = id_to_idx[cat_id]
            pixel_mask = coco.annToMask(ann)
            mask[pixel_mask == 1] = cls_idx

    return mask

def process_single_data(coco, img_id, img_dir, id_to_idx, transform=None):
    img_info = coco.loadImgs(img_id)[0]
    img_path = os.path.join(img_dir, img_info['file_name'])

    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    mask = create_mask_from_json(coco, img_id, img_info, id_to_idx)

    if transform:
        augmented = transform(image=image, mask=mask)
        image, mask = augmented['image'], augmented['mask']

    return image, torch.as_tensor(mask).long()

train_transform = A.Compose([
    #  원본 해상도에서 352x352 크기로 무작위 추출 (화질 저하 없음)
    A.RandomCrop(height=CFG['img_size'][0], width=CFG['img_size'][1], p=1.0),
    A.PadIfNeeded(min_height=CFG['img_size'][0], min_width=CFG['img_size'][1], p=1.0),

    # --- 야간 전용 Augmentation 추가 ---
    A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.RandomGamma(gamma_limit=(80, 120), p=0.5), # 어두운 저조도 개선
    A.GaussNoise(std_range=(0.02, 0.05), p=0.3), # 야간 노이즈 대응

    # --- 기하학적 변형 (데이터 수 보충용) ---
    A.HorizontalFlip(p=0.5), # 좌우 반전
    # 0.0625는 머신러닝/딥러닝 커뮤니티에서 오랫동안 검증된 '사실상 표준(De Facto Standard)
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5), # 이동/크기/회전

    # ImageNet 데이타셋의 평균값 나쁘지 않음. SegFormer가 ImageNet/Cityscapes로 배웠으니까
    # 모델이 새로운 사진을 받을 때: 입력_이미지 = (원본_이미지 - 평균) / 표준편차
    # 이렇게 계산해주면, 어떤 사진이 들어와도 "평균이 0이고 표준편차가 1인(Standard Normal Distribution)" 아주 예쁜 데이터로 변신
    # 전체 야간 데이터셋의 Mean/Std를 직접 계산한 값
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])

coco = COCO(JSON_PATH)
img_ids = list(coco.imgs.keys())
cat_ids = coco.getCatIds()
id_to_idx = {cat_id: i for i, cat_id in enumerate(cat_ids)}
print(f"Category Mapping: {id_to_idx}")

/Users/jamesyang/.pyenv/versions/3.12.2/envs/aipel/lib/python3.12/site-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


loading annotations into memory...
Done (t=0.91s)
creating index...
index created!
Category Mapping: {1329681: 0, 1323885: 1, 1323884: 2, 1323882: 3, 1323881: 4, 1323880: 5}


## Traing / Val 분

In [80]:
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class MotorcycleNightRideDataset(Dataset):
    def __init__(self, coco, img_ids, img_dir, id_to_idx, transform=None):
        self.coco = coco
        self.img_ids = img_ids
        self.img_dir = img_dir
        self.id_to_idx = id_to_idx
        self.transform = transform

    def __len__(self):
        return len(self.img_ids)

    def __getitem__(self, idx):
        img_id = self.img_ids[idx]
        image, mask = process_single_data(self.coco, img_id, self.img_dir, self.id_to_idx, self.transform)
        return image, mask

# 1. 데이터 로드 및 ID 분할 (8:2)
coco = COCO(JSON_PATH)
all_ids = list(coco.imgs.keys())
train_ids, val_ids = train_test_split(all_ids, test_size=0.2, random_state=42)

# 2. Transform 정의 (기존 정의 활용 및 Val용 추가)
val_transform = A.Compose([
    A.Resize(CFG['img_size'][0], CFG['img_size'][1]),
    A.Normalize(mean=(0.281, 0.268, 0.346), std=(0.347, 0.290, 0.292)),
    ToTensorV2()
])

# 3. 데이터셋 인스턴스 생성
train_dataset = MotorcycleNightRideDataset(coco, train_ids, IMG_DIR, id_to_idx, train_transform)
val_dataset = MotorcycleNightRideDataset(coco, val_ids, IMG_DIR, id_to_idx, val_transform)

# 4. 데이터 로더 생성
train_loader = DataLoader(
    train_dataset, 
    batch_size=CFG['batch_size'], 
    shuffle=True, 
    num_workers=CFG['num_worker'],
    pin_memory=True
)
val_loader = DataLoader(
    val_dataset, 
    batch_size=CFG['batch_size'], 
    shuffle=False, 
    num_workers=CFG['num_worker'],
    pin_memory=True
)

print(f"✅ Data Ready: Train={len(train_ids)}, Val={len(val_ids)}")

loading annotations into memory...
Done (t=0.85s)
creating index...
index created!
✅ Data Ready: Train=160, Val=40


### 📉 클래스별 분포 요약 (내림차순)

1. **Undrivable (주행 불가 영역)**: **42.9%** (압도적 1위)
   - 배경(하늘, 건물, 풀숲 등)이 이미지의 절반 가까이 차지합니다.
2. **Road (주행 가능 도로)**: **27.1%**
   - 도로 자체도 꽤 많은 영역을 차지합니다.
3. **My bike (내 오토바이)**: **15.8%**
   - 주행자 시점이라 내 오토바이가 항상 보이기 때문에 비율이 높습니다.
4. **Rider (탑승자)**: **8.1%**
   - 다른 오토바이 운전자나 내 신체가 포함된 것으로 보입니다.
5. **Moveable (이동 물체)**: **4.7%**
   - 다른 차량, 보행자 등 안전에 가장 중요한 장애물인데 비율이 매우 낮습니다.
6. **Lane Mark (차선)**: **1.4%**
   - 가장 심각한 불균형입니다. 도로 주행의 핵심인 차선이 고작 1% 남짓입니다.

In [81]:
id2label = {i: coco.loadCats(cat_id)[0]['name'] for cat_id, i in id_to_idx.items()}
label2id = {v: k for k, v in id2label.items()}

model = SegformerForSemanticSegmentation.from_pretrained(
    CFG['model_name'],
    num_labels=len(id_to_idx),
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True
).to(CFG['device'])

optimizer = torch.optim.AdamW(model.parameters(), lr=CFG['lr'], weight_decay=0.01)

# ⚖️ 클래스별 가중치 설정 (Class Weights)
weights = torch.tensor([
    5.0,   # Rider (8.1%) -> 적당히 높임
    2.0,   # My bike (15.8%) -> 낮춤 (이미 많음)
    10.0,  # Moveable (4.7%) -> 강력하게 높임
    20.0,  # Lane Mark (1.4%) -> 아주 강력하게!! (핵심)
    1.0,   # Road (27.1%) -> 기본
    0.5    # Undrivable (42.9%) -> 낮춤 (너무 많아서 방해됨)
], dtype=torch.float).to(CFG['device'])

criterion = nn.CrossEntropyLoss(weight=weights)

scaler = GradScaler('cuda') if CFG['device'] == 'cuda' else None

Loading weights:   0%|          | 0/380 [00:00<?, ?it/s]

SegformerForSemanticSegmentation LOAD REPORT from: nvidia/segformer-b2-finetuned-cityscapes-1024-1024
Key                           | Status   |                                                                                                    
------------------------------+----------+----------------------------------------------------------------------------------------------------
decode_head.classifier.bias   | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19]) vs model:torch.Size([6])                      
decode_head.classifier.weight | MISMATCH | Reinit due to size mismatch - ckpt: torch.Size([19, 768, 1, 1]) vs model:torch.Size([6, 768, 1, 1])

Notes:
- MISMATCH	:ckpt weights were loaded, but they did not match the original empty weight shapes.


In [ ]:
print("🚀 SegFormer-B2 Training Start...")

for epoch in range(CFG['epochs']):
    # --- Training Phase ---
    model.train()
    train_loss_sum = 0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]")
    
    for images, masks in pbar:
        X = images.to(CFG['device']).contiguous()
        y = masks.to(CFG['device']).contiguous()
        
        optimizer.zero_grad()
        
        # Mixed Precision 지원 (CUDA 전용)
        if CFG['device'] == 'cuda' and scaler:
            with torch.amp.autocast('cuda'):
                outputs = model(X).logits
                upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
                loss = criterion(upsampled_logits, y)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(X).logits
            upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
            loss = criterion(upsampled_logits, y)
            loss.backward()
            optimizer.step()
            
        train_loss_sum += loss.item()
        pbar.set_postfix(Loss=f"{loss.item():.4f}")
    
    avg_train_loss = train_loss_sum / len(train_loader)

    # --- Validation Phase ---
    model.eval()
    val_loss_sum = 0
    with torch.no_grad():
        for images, masks in val_loader:
            X = images.to(CFG['device']).contiguous()
            y = masks.to(CFG['device']).contiguous()
            
            outputs = model(X).logits
            upsampled_logits = nn.functional.interpolate(outputs, size=y.shape[-2:], mode="bilinear", align_corners=False)
            loss = criterion(upsampled_logits, y)
            val_loss_sum += loss.item()
            
    avg_val_loss = val_loss_sum / len(val_loader)
    print(f"📝 Epoch [{epoch+1}/{CFG['epochs']}] Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

🚀 SegFormer-B2 Training Start...


Epoch 1 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]

📝 Epoch [1/25] Train Loss: 1.6318 | Val Loss: 1.5289


Epoch 2 [Train]:   0%|          | 0/40 [00:00<?, ?it/s]